In [15]:
%pip install graphrag_sdk==0.1.3b0
%pip install -e /workspaces/autogen

1136172.97s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


1136178.53s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///workspaces/autogen
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for autogen (pyproject.toml) ... done
  Created wheel for autogen: filename=autogen-0.3.0-0.editable-py3-none-any.whl size=16273 sha256=20d08c707c24d67ad0f271f71cef54b81aa54a044da58030e67204fc994ee53b
  Stored in directory: /tmp/pip-ephem-wheel-cache-038ayx1o/wheels/d4/86/1f/a538740a449c67aa3ff8343698c29d70fc94236f70bde92144
Successfully built autogen
  Attempting uninstall: autogen
    Found existing installation: autogen 0.3.0
    Uninstalling autogen-0.3.0:
      Successfully uninstalled autogen-0.3.0

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the ke

In [8]:
import os

import autogen

config_list = autogen.config_list_from_json(env_or_file="../OAI_CONFIG_LIST")
os.environ["OPENAI_API_KEY"] = config_list[0]["api_key"]

In [10]:
from graphrag_sdk.schema import Schema

from autogen.agentchat.contrib.graph_rag.document import Document, DocumentType
from autogen.agentchat.contrib.graph_rag.falkor_graph_query_engine import FalkorGraphQueryEngine

# Auto generate graph schema from unstructured data
input_path = "../test/agentchat/contrib/graph_rag/the_matrix.txt"

movie_schema = Schema()
actor = movie_schema.add_entity("Actor").add_attribute("name", str, unique=True)
movie = movie_schema.add_entity("Movie").add_attribute("title", str, unique=True)
movie_schema.add_relation("ACTED", actor, movie)

query_engine = FalkorGraphQueryEngine(
    name="IMDB",
    host="172.18.0.3",
    port=6379,
    schema=movie_schema,
)

input_documents = [Document(doctype=DocumentType.TEXT, path_or_url=input_path)]

query_engine.init_db(input_doc=input_documents)

In [9]:
# Query your data
question = "Name a few actors who've played in 'The Matrix'"
answer = query_engine.query(question)
print(f"Answer: {answer}")

Answer: FalkorGraphQueryResult(answer="A few actors who've played in 'The Matrix' are:\n\n- Keanu Reeves\n- Laurence Fishburne\n- Carrie-Anne Moss\n- Hugo Weaving", results=[], messages=[{'role': 'system', 'content': 'You are a graph database expert fluent in the Cypher query language\n                with access to the following knowledge graph: The knowladge graph schema is as follows:\nThe graph contains the following node labels:\nMovie\nActor\nThe Movie label is associated with the following attribute(s):\ntitle which is of type str\nThe Actor label is associated with the following attribute(s):\nname which is of type str\nThe graph contains the following relationship types:\nACTED\nThe ACTED relationship connects the following labels:\nActor is connected via ACTED to Movie\nThis is the end of the knowladge graph schema description.\n'}, {'role': 'user', 'content': "Name a few actors who've played in 'The Matrix'"}, ChatCompletionMessage(content=None, refusal=None, role='assistant

In [16]:
from autogen import UserProxyAgent
from autogen.agentchat.contrib.graph_rag.falkor_graph_rag_capability import FalkorGraphRagCapability

graph_rag_agent = UserProxyAgent(
    name="user_proxy",
    code_execution_config=False,
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
    human_input_mode="ALWAYS",
)
graph_rag_capability = FalkorGraphRagCapability(query_engine)
graph_rag_capability.add_to_agent(graph_rag_agent)

graph_rag_agent.initiate_chat(graph_rag_agent, message="Name a few actors who've played in 'The Matrix'")

user_proxy (to user_proxy):

Name a few actors who've played in 'The Matrix'

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': "Name a few actors who've played in 'The Matrix'", 'role': 'assistant', 'name': 'user_proxy'}, {'content': "The following actors have played in 'The Matrix':\n\n- Keanu Reeves\n- Laurence Fishburne\n- Carrie-Anne Moss\n- Hugo Weaving", 'role': 'user', 'name': 'user_proxy'}], summary="The following actors have played in 'The Matrix':\n\n- Keanu Reeves\n- Laurence Fishburne\n- Carrie-Anne Moss\n- Hugo Weaving", cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=['exit'])